In [3]:
import OpenDartReader
import pandas as pd 
import os
import glob

# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'

dart = OpenDartReader(api_key)

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': str, 
                                                'stock_code': str})
df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
df.head()

,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020
3,케이티,A+,"33,662,545,000,000",00190321,030200,2020
4,SK,A+,"137,667,906,000,000",00181712,034730,2020


In [14]:
# def intang(code, year):# stock_code
#     lst = dart.list(code, start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
    
#     # 변경1: 원하는 년도의 사업 보고서 고르기 
#     report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(year)))].iloc[0, 5]
#     url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
#     html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1] # url 주소
    
#     docu = pd.read_html(html) # 데이터 읽어오기 
    
#     fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'})
#     intangible = fin_state[fin_state['tag'].str.contains('무형자산')] # 무형자산 DF
#     output = intangible.iloc[0, 1]

#     return output

In [44]:
import numpy as np
entire_codes = []
for f in glob.glob('./2020/fin_2020/*.csv'):
    code = f.split('\\')[1][:6]
    entire_codes.append(code)

entire_codes
code_missing = [code for code in list(df['stock_code'].unique()) if code not in entire_codes]

print('저장된 회사 수: ', len(entire_codes)) # 847개 중 690개 # 157개 누락
print('누락 회사 수: ', len(code_missing)) # 847개 중 177개 누락


# for index in range(df.shape[0]):
#     code, year = df.loc[index, 'stock_code'], df.loc[index, 'year']
#     try:
#         df.loc[index, 'intangible'] = intang(code, year)
#     except:
#         df.loc[index, 'intangible'] = 0.0
# df

저장된 회사 수:  670
누락 회사 수:  177


In [45]:
df_c = df.copy()
df_c['intangible'] = [np.nan] * df_c.shape[0]
df_c['ratio'] = [np.nan] * df_c.shape[0]

missing_codes_intang = []
for code in df_c['stock_code'].unique():
    try:
        docu = pd.read_csv(f'./2022/fin_2022/{code}_2022.csv', index_col=0)
        intang = docu[docu['tag'].str.contains('무형자산')].iloc[0, 1:]
        
        data = df_c[df_c['stock_code'] == code].sort_values(['year'], ascending=False)

        for i, (k,v) in enumerate(data.intangible.items()):
            df_c.loc[k, 'intangible'] = intang[i]
    except:
        missing_codes_intang.append(code)

df_c['intangible'] = pd.to_numeric(df_c['intangible'])

In [54]:
# for code in missing_codes_intang:
#     dart.list()
    
lst = dart.list('009830', start='2019', end='2024') # 2019-2024 모든 보고서에서 데이터 찾아오기
    
# 변경1: 원하는 년도의 사업 보고서 고르기 
report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(2022)))].iloc[0, 5]
url = dart.sub_docs(report_num) #사업보고서 번호를 통해 url 요청
html = url[url['title'].str.contains('연결재무제표')] # url 주소

,title,url
0,정 정 신 고 (보고),http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1,영 업 보 고 서,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [55]:
url.iloc[1, 1]

'http://dart.fss.or.kr/report/viewer.do?rcpNo=20230316001209&dcmNo=9066248&eleId=2&offset=2120&length=7609&dtd=dart3.xsd'

In [25]:
code = '00102113'
docu = pd.read_csv(f'./2022/cash_2022/{code}_2022.csv', index_col=0)
docu[docu['se'].str.contains('현금배당성향')].iloc[0, 2:].values

df_c[df_c['corp_code']==code].sort_values(['year'], ascending=False)

,corp_name,esg,fin_state,corp_code,stock_code,year,intangible,ratio
2335,경인양행,C,"548,252,999,063",00102113,012610,2022,5.277820e+09,0.0
1488,경인양행,C,"579,492,465,098",00102113,012610,2021,6.420705e+09,0.0
641,경인양행,C,"505,102,150,581",00102113,012610,2020,7.040427e+09,0.0


In [31]:
missing_cash = []
for code in df_c['corp_code'].unique():
    try:
        docu = pd.read_csv(f'./2022/cash_2022/{code}_2022.csv', index_col=0)
        cash = docu[docu['se'].str.contains('현금배당성향')].iloc[0, 2:].values
        
        data = df_c[df_c['corp_code'] == code].sort_values(['year'], ascending=False)

        for i, (k,v) in enumerate(data.ratio.items()):
            df_c.loc[k, 'ratio'] = cash[i]
    except:
        missing_cash.append(code)

In [33]:
# df_c = df_c['ratio'].replace('-', np.nan)

In [34]:
# df_c.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2541 entries, 0 to 2540
Series name: ratio
Non-Null Count  Dtype 
--------------  ----- 
1642 non-null   object
dtypes: object(1)
memory usage: 20.0+ KB
